# Transformer
完全基于注意力机制，没有任何CNN layer或RNN layer

## 1. 模型
### 1.1 编码器（encoder）
多个相同层汇聚而成，每个层都有两个子层（sublayer）。
- 第一个子层是多头自注意力（multihead self-attention）汇聚；
- 第二个子层是基于位置的前馈神经网络（positionwise feed-forward network）。
- 在计算自注意力时，查询、键和值都来自前一个编码器**层**的输出。
- 每个子层使用残差连接（residual connection）。
- 对于序列中任意位置的任何输入$\mathbf{x}\in\mathbb{R}^d$，都要求$\mathrm{sublayer}(\mathbf{x})\in\mathbb{R}^d$，以便残差连接满足$\mathbf{x}+\mathrm{sublayer}(\mathbf{x})\in\mathbb{R}^d$
- 在残差连接加法后，紧接着应用层规范化（layer normalization）。
- 因此对于输入序列对应的每个位置，transformer编码器都将输出一个$d$维表示向量
### 1.2 解码器（decoder）
多个相同层叠加而成。每层有三个子层（sublayer）
- 第一个子层是